### Recolher dados da Web

Neste notebook vamos sistematizar a recolha de dados a partir do site do Genbank.

Dado uma determinada sequência, identificada por um id, por exemplo, quer-se ir ao site descarregar o respetivo registo, tratá-lo (somo já fizemos) e depois inserir as partes relevantes em base de dados.

Exemplos de links para sequências:
- https://www.ncbi.nlm.nih.gov/nuccore/L42022
- https://www.ncbi.nlm.nih.gov/nuccore/L42023
- https://www.ncbi.nlm.nih.gov/nuccore/LC740868.1

Depois de se pedir este link, dentro da página, em Javascript, é feito um outro pedido ao servidor a pedir o record da sequência.

Exemplo:
- https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=genbank&id=804715&conwithfeat=on&hide-cdd=on&ncbi_phid=null

No exemplo seguinte,. pede-se a página, mas a mesma não contém o registo que nos interessa.

O registo é carregado assincronamente, através de programação Javascript.

In [1]:
import requests
r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/PA500505.1')
print(r.content)

b'<?xml version="1.0" encoding="utf-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\n    <head xmlns:xi="http://www.w3.org/2001/XInclude"><meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <!-- meta -->\n    <meta name="robots" content="index,nofollow,noarchive" />\n<meta name="ncbi_app" content="entrez" /><meta name="ncbi_db" content="nuccore" /><meta name="ncbi_report" content="genbank" /><meta name="ncbi_format" content="html" /><meta name="ncbi_pagesize" content="20" /><meta name="ncbi_sortorder" content="default" /><meta name="ncbi_pageno" content="1" /><meta name="ncbi_resultcount" content="1" /><meta name="ncbi_op" content="retrieve" /><meta name="ncbi_pdid" content="genbank" /><meta name="ncbi_sessionid" content="CE8AABB338FBC7D1_0468SID" /><meta name="ncbi_uidlist" content="2246533317" /><meta name="ncbi_fi

No exemplo seguinte,. em vez de se pedir a página que contém o registo, pede-se apenas o registo, depois de percebermos como o mesmo é pedido.

In [25]:
# pip3 install html5lib

import requests
from bs4 import BeautifulSoup

# Making a GET request
r = requests.get('https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=2246533317&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000')
# r = requests.get('https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=804715&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=html&ncbi_phid=CE8B6449389BE9F100000000068605EB&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000')

print( r.content ) 




b'LOCUS       PA500505                 318 bp    DNA     linear   PAT 31-MAY-2022\nDEFINITION  JP 2022506561-A/6: ANTIBODY BINDING TO HUMAN IL-1beta, PREPARATION\n            METHOD THEREFOR AND USE THEREOF.\nACCESSION   PA500505\nVERSION     PA500505.1\nKEYWORDS    JP 2022506561-A/6.\nSOURCE      Homo\n  ORGANISM  Homo\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae.\nREFERENCE   1  (bases 1 to 318)\n  AUTHORS   Xia,Y., Wang,Z., Zhang,P. and Li,B.\n  TITLE     ANTIBODY BINDING TO HUMAN IL-1beta, PREPARATION METHOD THEREFOR AND\n            USE THEREOF\n  JOURNAL   Patent: JP 2022506561-A 6 17-JAN-2022;\n            ZEDA BIOPHARMACEUTICALS INC\nCOMMENT     OS   Homo\n            PN   JP 2022506561-A/6\n            PD   17-JAN-2022\n            PF   04-NOV-2019 JP 2021523978\n            PR   07-NOV-2018 CN 201811322002.7\n            PA   ZEDA BIOP

### Problema

Ao pedir a página, não vem a registo da sequência.

Para pedir o registo, o mesmo pede-se por um id (um número interno) que é diferente do id da sequência (L42022, por exemplo).

### Solução

A solução passa por fazer dois pedidos. No primeiro, pede-se a página e extrai-se apenas o id numérico interno, associado à sequência. Esse id interno é então usado para se fazer o segundo pedido.



In [1]:
# pip3 install html5lib

import requests
from bs4 import BeautifulSoup

# Making a GET request
r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/L42022')
# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')

# Procurar um tag meta com um determinado atributo
lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

id = ""
url = ""
for line in lines:
	# print(line)
	# if 'name' in line.attrs:
	# 	print(line.attrs['name'])
	if 'content' in line.attrs:
		# print(line.attrs['content'])		
		id = line.attrs['content']

if id:
	url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

r2 = requests.get( url )

print( r2.content )

b'LOCUS       HIVI5C                   231 bp    DNA     linear   VRL 24-MAR-1997\nDEFINITION  Human immunodeficiency virus type 1 (isolate genotype C, I5) gag\n            gene, partial cds.\nACCESSION   L42022\nVERSION     L42022.1\nKEYWORDS    gag gene; p24 protein.\nSOURCE      Human immunodeficiency virus 1 (HIV-1)\n  ORGANISM  Human immunodeficiency virus 1\n            Viruses; Riboviria; Pararnavirae; Artverviricota; Revtraviricetes;\n            Ortervirales; Retroviridae; Orthoretrovirinae; Lentivirus.\nREFERENCE   1  (bases 1 to 231)\n  AUTHORS   Voevodin,A., Crandall,K.A., Seth,P. and al Mufti,S.\n  TITLE     HIV type 1 subtypes B and C from new regions of India and Indian\n            and Ethiopian expatriates in Kuwait\n  JOURNAL   AIDS Res. Hum. Retroviruses 12 (7), 641-643 (1996)\n   PUBMED   8743090\nFEATURES             Location/Qualifiers\n     source          1..231\n                     /organism="Human immunodeficiency virus 1"\n                     /proviral\n   

In [7]:
import requests
from bs4 import BeautifulSoup
import time


for i in range(200,220):
	# Making a GET request
	r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/L42{}'.format(i))
	time.sleep(1)
	# Parsing the HTML
	soup = BeautifulSoup(r.content, 'html.parser')

	# Procurar um tag meta com um determinado atributo
	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])
			id = line.attrs['content']

	if id:
		url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

	r2 = requests.get( url )

	print( r2.content )

b'LOCUS       HLICPITSAD               249 bp    DNA     linear   PLN 07-JUL-2003\nDEFINITION  Heliamphora minor chloroplast internal transcribed spacer 2,\n            partial sequence.\nACCESSION   L42200\nVERSION     L42200.2\nKEYWORDS    .\nSOURCE      chloroplast Heliamphora minor\n  ORGANISM  Heliamphora minor\n            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;\n            Spermatophyta; Magnoliopsida; eudicotyledons; Gunneridae;\n            Pentapetalae; asterids; Ericales; Sarraceniaceae; Heliamphora.\nREFERENCE   1  (bases 1 to 249)\n  AUTHORS   Bayer,R.J., Hufford,L. and Soltis,D.E.\n  TITLE     Phylogenetic relationships in Sarraceniaceae based on rbcL and ITS\n            sequences\n  JOURNAL   Syst. Bot. 21 (2), 121-134 (1996)\nREFERENCE   2  (bases 1 to 249)\n  AUTHORS   Bayer,R.J.\n  TITLE     The implications of morphological and molecular sequence data from\n            cpDNA and nrDNA for phylogeny reconstruction in Sarraceniaceae\n  JOUR